In [64]:
import pandas as pd
import snappy
import os
from itertools import groupby
import pickle
import numpy as np

In [65]:
#option d'affichage pour les df
pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [66]:
#choix des colonnes pour l'étude
col = ['time', 'unknown_IMISSIONTRAINNUMBER', 'WC_CAR01_LCST_ISYSTEMLIFESIGN', 'WC_CAR03_LCST_ISYSTEMLIFESIGN', 'WC_CAR05_LCST_ISYSTEMLIFESIGN', 'WC_CAR07_LCST_ISYSTEMLIFESIGN', 'WC_CAR01_LCST_IWSUTANKLEVEL',
'WC_CAR03_LCST_IWSUTANKLEVEL', 'WC_CAR05_LCST_IWSUTANKLEVEL', 'WC_CAR07_LCST_IWSUTANKLEVEL', 'WC_CAR01_LCST_IFWTANKCONTENT', 'WC_CAR03_LCST_IFWTANKCONTENT', 'WC_CAR05_LCST_IFWTANKCONTENT',
'WC_CAR07_LCST_IFWTANKCONTENT', 'WC_CAR01_LCST_IWWTANKCONTENT', 'WC_CAR03_LCST_IWWTANKCONTENT', 'WC_CAR05_LCST_IWWTANKCONTENT', 'WC_CAR07_LCST_IWWTANKCONTENT', 'WC_CAR01_LCST_IWCWORKTIME',
'WC_CAR03_LCST_IWCWORKTIME', 'WC_CAR05_LCST_IWCWORKTIME', 'WC_CAR07_LCST_IWCWORKTIME', 'WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE',
'WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR01_LCST_IDOORCYCLCNT', 'WC_CAR03_LCST_IDOORCYCLCNT', 'WC_CAR05_LCST_IDOORCYCLCNT',
'WC_CAR07_LCST_IDOORCYCLCNT', 'WC_CAR01_LCST_IEJECTORCNT', 
'WC_CAR03_LCST_IEJECTORCNT', 'WC_CAR05_LCST_IEJECTORCNT', 'WC_CAR07_LCST_IEJECTORCNT',  'WC_CAR01_LCST_IWATERTAPCNT', 'WC_CAR03_LCST_IWATERTAPCNT', 'WC_CAR05_LCST_IWATERTAPCNT', 
'WC_CAR07_LCST_IWATERTAPCNT', 'WC_CAR01_LCST_IDELIMINGCNT', 'WC_CAR03_LCST_IDELIMINGCNT', 'WC_CAR05_LCST_IDELIMINGCNT', 'WC_CAR07_LCST_IDELIMINGCNT', 'WC_CAR01_LCST_IDEFROSTCNT', 
'WC_CAR03_LCST_IDEFROSTCNT', 'WC_CAR05_LCST_IDEFROSTCNT', 'WC_CAR07_LCST_IDEFROSTCNT', 'WC_CAR01_LCST_IFWT05CNT', 'WC_CAR03_LCST_IFWT05CNT', 'WC_CAR05_LCST_IFWT05CNT', 'WC_CAR07_LCST_IFWT05CNT',
'WC_CAR01_LCST_IWWT95CNT', 'WC_CAR03_LCST_IWWT95CNT', 'WC_CAR05_LCST_IWWT95CNT', 'WC_CAR07_LCST_IWWT95CNT', 'WC_CAR01_LCST_FWCOUTOFSERVICE', 'WC_CAR03_LCST_FWCOUTOFSERVICE', 
'WC_CAR05_LCST_FWCOUTOFSERVICE', 'WC_CAR07_LCST_FWCOUTOFSERVICE', 'WC_CAR01_LCST_IWCREADY', 'WC_CAR03_LCST_IWCREADY', 'WC_CAR05_LCST_IWCREADY', 'WC_CAR07_LCST_IWCREADY', 
'WC_CAR01_LCST_FFWTEMPTY', 'WC_CAR03_LCST_FFWTEMPTY', 'WC_CAR05_LCST_FFWTEMPTY', 'WC_CAR07_LCST_FFWTEMPTY', 'WC_CAR01_LCST_IFLUSHCYCCNT', 'WC_CAR03_LCST_IFLUSHCYCCNT', 'WC_CAR05_LCST_IFLUSHCYCCNT', 'WC_CAR07_LCST_IFLUSHCYCCNT',
'WC_CAR01_LCST_IDRAINCOMMUTATION', 'WC_CAR03_LCST_IDRAINCOMMUTATION', 'WC_CAR05_LCST_IDRAINCOMMUTATION', 'WC_CAR07_LCST_IDRAINCOMMUTATION']


In [67]:
#adresse des données
REP_DATA = 'D:/ESTACA/4A/Projet industriel/Données/Données_2023_03_26'

In [68]:
#liste des répertoires de données (contenant les fichiers parquet)
content_list = os.listdir(REP_DATA)

In [69]:
#création d'un dictionnaire pour les noms de rames (clé = rame, valeur = liste des répertoires)
#on ne garde que la premiere partie du nom des fichiers
racine_dict = {}
for racine, group_rep in groupby(content_list, lambda nom: nom.split('_')[0]):
    racine_dict[racine] = list(group_rep)

In [70]:
racine_dict

{'z5500503': ['z5500503_20230103_173926_0',
  'z5500503_20230306_182130_0',
  'z5500503_20230309_165000_0',
  'z5500503_20230309_192913_2',
  'z5500503_20230310_031902_0',
  'z5500503_20230310_134536_0',
  'z5500503_20230310_165212_0',
  'z5500503_20230310_195415_1',
  'z5500503_20230311_025455_2',
  'z5500503_20230311_053834_1',
  'z5500503_20230311_062038_2',
  'z5500503_20230311_095646_0',
  'z5500503_20230311_113830_1',
  'z5500503_20230313_093146_0',
  'z5500503_20230313_151914_0',
  'z5500503_20230313_164909_0',
  'z5500503_20230313_172345_1',
  'z5500503_20230313_193228_0',
  'z5500503_20230313_200229_1',
  'z5500503_20230313_211931_3',
  'z5500503_20230313_221959_0',
  'z5500503_20230313_225006_1',
  'z5500503_20230313_232511_2',
  'z5500503_20230314_000701_3',
  'z5500503_20230314_010954_0',
  'z5500503_20230314_014202_1',
  'z5500503_20230314_043104_0',
  'z5500503_20230314_050351_1',
  'z5500503_20230314_070414_0',
  'z5500503_20230314_130139_0',
  'z5500503_20230314_143101_

In [71]:
def cnt_missions(df):
    dg = df.copy()
    dg['nombre_mission'] = 0
    dg['cpt_mission']=0
    mission = 0
    for t in range(1, len(df)):
        if df.unknown_IMISSIONTRAINNUMBER[t]!=df.unknown_IMISSIONTRAINNUMBER[t-1]:
            dg['nombre_mission'][t] = 1
            mission +=1
        dg['cpt_mission'][t]= mission
    return dg

In [72]:
def remplissage_WSU(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_remplissageWSU'] = 0
        for t in range(0, len(df)-1):
            if df['WC_'+voiture+'_LCST_IWSUTANKLEVEL'][t]<df['WC_'+voiture+'_LCST_IWSUTANKLEVEL'][t+1]:
                df['WC_'+voiture+'_LCST_remplissageWSU'][t] += 1

In [73]:
def consommation(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_consommation'] = df['WC_'+voiture+'_LCST_IWATERTAPCNT']*0.4+1.14*df['WC_'+voiture+'_LCST_remplissageWSU']

In [74]:
def remplissage_instantane(df):
    for voiture in ['CAR01', 'CAR03', 'CAR05', 'CAR07']:
        df['WC_'+voiture+'_LCST_remplissageInstantané%'] = (df['WC_'+voiture+'_LCST_IFLUSHCYCCNT']*0.8*100)/315

In [75]:
#création d'un dictionnaire de df en fonction des rames (clé = rame, valeur = df de données)
#lecture de tous les fichiers qui se rapportent à la même rame
#suppression de la dernière ligne car que des NaN dedans
#conversion du code mission en int pour que les 16 chiffres s'affichent
dict_df = {}
for rac, reps in racine_dict.items():
    df_list = []
    for rep in reps:
        df_temp = pd.read_parquet(REP_DATA + '/' + rep + '/TT_IP.parquet')
        df_temp = df_temp.loc[:, col].iloc[:-1]
        df_temp.unknown_IMISSIONTRAINNUMBER = df_temp.unknown_IMISSIONTRAINNUMBER.astype(np.int64)
        df_temp = df_temp.rename(columns={"time":'x_time'})
        df_list.append(df_temp)
        df_temp.drop(df_temp.loc[df_temp['unknown_IMISSIONTRAINNUMBER']==0].index, inplace=True)
    df_concat = pd.concat(df_list, ignore_index=True)
    df_concat = cnt_missions(df_concat)
    remplissage_WSU(df_concat)
    consommation(df_concat)
    remplissage_instantane(df_concat)
    df_concat['jour']=df_concat.x_time.dt.date
    df_concat['rame']=rac
    dict_df[rac] = df_concat

C:\Users\canel\AppData\Local\Temp\ipykernel_20132\3792332642.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dg['cpt_mission'][t]= mission
C:\Users\canel\AppData\Local\Temp\ipykernel_20132\3792332642.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dg['nombre_mission'][t] = 1
C:\Users\canel\AppData\Local\Temp\ipykernel_20132\1024141870.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['WC_'+voiture+'_LCST_remplissageWSU'][

In [76]:
rep

'z5500625_20230316_074855_0'

In [77]:
#sauvegarde du dictionnaire de df des rames en pickle
with open('dict_df.pkl', 'wb') as file:      
    pickle.dump(dict_df, file)

In [78]:
#création du dictionnaire de df en fonction des missions de chaque rame (clé : rame, valeur : dictionnaire(clé = mission, valeur = df de données))
#lecture de toutes les missions possible d'une rame
dict_missions = {}
for rame, datas in dict_df.items():
    dict_mission_rame = {}
    missions = datas.unknown_IMISSIONTRAINNUMBER.unique()
    for mission in missions :
        df_temp = datas.query('unknown_IMISSIONTRAINNUMBER == @mission') #@ car mission variable externe a df
        dict_mission_rame[mission] = df_temp
    dict_missions[rame]=dict_mission_rame

In [79]:
#sauvegarde du dictionnaire des missions en fonction des rames dans un pickle
with open('dict_rame_missions.pkl', 'wb') as file:      
    pickle.dump(dict_missions, file)

In [80]:
#sauvegarde du dictionnaire des missions en fonction des rames dans un pickle
df_complet = pd.concat(dict_df.values())
with open('df_complet.pkl', 'wb') as file:      
    pickle.dump(df_complet, file)

In [81]:
list(dict_df.keys())

['z5500503',
 'z5500517',
 'z5500523',
 'z5500529',
 'z5500533',
 'z5500535',
 'z5500545',
 'z5500577',
 'z5500579',
 'z5500589',
 'z5500601',
 'z5500613',
 'z5500625']